# Analysis of A05 results

## Data loading

In [ ]:
from curve_analyzer.utils.data_processing import load_curves, load_trait, filter_df
curves = load_curves() # load all curves
df_a05 = load_trait("a05") # load all results for trait a05
df_a05 = df_a05.merge(curves, "left", "curve") # join curve metadata to trait results

## Data filtering

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, HBox, VBox

style = {'description_width': 'initial'}

def bitlength_choice(bitlengths=widgets.SelectMultiple(
        options=[112, 128, 160, 192, 224, 256],
        value=[256],
        description='bit length:',
        layout=widgets.Layout(width='120px', height='120px')
        )):
    return bitlengths

def l_choice(ls=widgets.SelectMultiple(
        options=[2, 3, 5, 7],
        value=[3],
        description='l:',
        layout=widgets.Layout(width='110px', height='auto'),
        )):
    return ls

def max_cofactor_choice(max_cofactor=widgets.IntSlider(
    min=1, max=100, step=1, value=1,
    style=style,
    description='max cofactor:'
    )):
    return max_cofactor

chosen = list(map(interactive, [bitlength_choice, l_choice, max_cofactor_choice]))
# [interactive(bitlength_choice), interactive(l_choice), in]
HBox(chosen, layout=widgets.Layout(justify_content='space-evenly'))

## Data inspection

In [ ]:
bitlengths, ls, max_cofactor = [param.result for param in chosen]
df = filter_df(df_a05, bitlength=bitlength, sources=["nist"], max_cofactor=max_cofactor)
df = df[df["l"].isin(ls)]
df

In [ ]:
df.agg(set) # display only distinct values

In [ ]:
df.describe()

## Data visualization

In [ ]:
from curve_analyzer.utils.visualization import normalized_barplot, normalized_bubbleplot

In [ ]:
for column in ("least", "full"):
    normalized_barplot(df, column, title=f"Normalized barplot of {column}")

In [ ]:
for l in (2, 3):
    normalized_bubbleplot(df[df["l"] == l], f"least", f"full", title=f"Normalized bubble plot for l={l}")

## Playground